In [1]:
import pandas as pd
from datetime import datetime, timedelta
import calendar

In [2]:
def generate_calendar_table(start_date='2014-01-01', end_date='2026-01-01'):
    """
    Generate a comprehensive calendar table with various date-related columns
    """
    # Create date range
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # Create base dataframe
    df = pd.DataFrame({'datetime': date_range})
    
    # Basic date components
    df['date'] = df['datetime'].dt.date
    df['year'] = df['datetime'].dt.year
    df['month'] = df['datetime'].dt.month
    df['day'] = df['datetime'].dt.day
    df['year_month'] = df['datetime'].dt.to_period('M')
    
    # Day of week (0=Monday, 6=Sunday)
    df['day_of_week_num'] = df['datetime'].dt.dayofweek
    df['day_of_week'] = df['datetime'].dt.day_name()
    df['day_of_week_abbr'] = df['datetime'].dt.strftime('%a')
    
    # Week number (ISO week)
    df['week_number'] = df['datetime'].dt.isocalendar().week
    df['iso_year'] = df['datetime'].dt.isocalendar().year
    
    # Month information
    df['month_name'] = df['datetime'].dt.month_name()
    df['month_abbr'] = df['datetime'].dt.strftime('%b')
    df['days_in_month'] = df['datetime'].dt.days_in_month
    
    # Quarter information
    df['quarter'] = df['datetime'].dt.quarter
    df['year_quarter'] = df['year'].astype(str) + '-Q' + df['quarter'].astype(str)
    
    # Day of year
    df['day_of_year'] = df['datetime'].dt.dayofyear
    
    # Weekend/weekday flags
    df['is_weekend'] = df['day_of_week_num'].isin([5, 6])  # Saturday=5, Sunday=6
    df['is_weekday'] = ~df['is_weekend']
    
    # Beginning/end of period flags
    df['is_month_start'] = df['datetime'].dt.is_month_start
    df['is_month_end'] = df['datetime'].dt.is_month_end
    df['is_quarter_start'] = df['datetime'].dt.is_quarter_start
    df['is_quarter_end'] = df['datetime'].dt.is_quarter_end
    df['is_year_start'] = df['datetime'].dt.is_year_start
    df['is_year_end'] = df['datetime'].dt.is_year_end
    
    # Fiscal year (assuming fiscal year starts in April)
    df['fiscal_year'] = df.apply(lambda x: x['year'] if x['month'] >= 4 else x['year'] - 1, axis=1)
    df['fiscal_quarter'] = df.apply(
        lambda x: ((x['month'] - 4) // 3 + 1) if x['month'] >= 4 else ((x['month'] + 8) // 3 + 1), 
        axis=1
    )
    
    # Leap year flag
    df['is_leap_year'] = df['datetime'].dt.is_leap_year
    
    # Season (Northern Hemisphere)
    def get_season(month):
        if month in [12, 1, 2]:
            return 'Winter'
        elif month in [3, 4, 5]:
            return 'Spring'
        elif month in [6, 7, 8]:
            return 'Summer'
        else:
            return 'Fall'
    
    df['season'] = df['month'].apply(get_season)
    
    # Business day flag (excludes weekends, but not holidays)
    df['is_business_day'] = df['datetime'].dt.dayofweek < 5
    
    # Week of month
    df['week_of_month'] = df['datetime'].apply(
        lambda x: (x.day - 1) // 7 + 1
    )
    
    # Days from epoch (useful for calculations)
    epoch = pd.Timestamp('1970-01-01')
    df['days_from_epoch'] = (df['datetime'] - epoch).dt.days
    
    # Formatted date strings
    df['date_iso'] = df['datetime'].dt.strftime('%Y-%m-%d')
    df['date_us'] = df['datetime'].dt.strftime('%m/%d/%Y')
    df['date_eu'] = df['datetime'].dt.strftime('%d/%m/%Y')
    df['date_readable'] = df['datetime'].dt.strftime('%B %d, %Y')
    
    # Sort by datetime to ensure proper order
    df = df.sort_values('datetime').reset_index(drop=True)
    
    return df

In [3]:
calendar_df = generate_calendar_table('2014-01-01', '2026-01-01')

In [4]:
print(f"Calendar table generated with {len(calendar_df)} rows")
print(f"Date range: {calendar_df['datetime'].min()} to {calendar_df['datetime'].max()}")
print("\nColumns in the calendar table:")
for i, col in enumerate(calendar_df.columns, 1):
    print(f"{i:2d}. {col}")

Calendar table generated with 4384 rows
Date range: 2014-01-01 00:00:00 to 2026-01-01 00:00:00

Columns in the calendar table:
 1. datetime
 2. date
 3. year
 4. month
 5. day
 6. year_month
 7. day_of_week_num
 8. day_of_week
 9. day_of_week_abbr
10. week_number
11. iso_year
12. month_name
13. month_abbr
14. days_in_month
15. quarter
16. year_quarter
17. day_of_year
18. is_weekend
19. is_weekday
20. is_month_start
21. is_month_end
22. is_quarter_start
23. is_quarter_end
24. is_year_start
25. is_year_end
26. fiscal_year
27. fiscal_quarter
28. is_leap_year
29. season
30. is_business_day
31. week_of_month
32. days_from_epoch
33. date_iso
34. date_us
35. date_eu
36. date_readable


In [6]:
calendar_df.to_csv('calendar_table.csv', index=False)